# Grab PlumX Data

## Request Functions
This part contains functions we need to fetch the web data and should also handle the exceptions while fetching here.

In [1]:
from grab_util import *
import json
import pandas as pd
import numpy as np
import math
import gc
import threading


In [2]:
origin_folder = 'SUSTC_Journals/articles_all'

## Grab PlumX from API

In [ ]:
# all_articles_df = pd.read_excel(f'{origin_folder}/all.xlsx', usecols=['SO', 'DI'])
all_articles_df = pd.read_csv(f'{origin_folder}/all.csv', usecols=['SO', 'DI'])

In [ ]:
plumx_info = {}

In [ ]:
def grab_plumx_info(startIndex, endIndex):
    global plumx_info
    all_articles_df_slice = all_articles_df[start_index:end_index]
    for index, doi in enumerate(all_articles_df_slice['DI']):
        if plumx_info.__contains__(doi):
            continue
        if index % 100 == 0:
            print(index)
        res = grab_from_url_json('https://plu.mx/api/v1/artifact/doi/' + str(doi))
        plumx_info[doi] = res

    print('plumx_info length:', len(plumx_info))

In [ ]:
class grabApiThread(threading.Thread):
    def __init__(self, threadID, name, start_index, end_index):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.start_index = start_index
        self.end_index = end_index
    def run(self):
        grab_plumx_info(self.start_index, self.end_index)


In [ ]:
%%time
step = 500
threads = []
for i in range(0, 20):
    plumx_info = {}
    for j in range(0, (int)(len(all_articles_df) / (step * 20))):
        start_index = i * (int)(len(all_articles_df)/step)*(int)(step/20) + j * step
        end_index = start_index + step
        if end_index + step > len(all_articles_df):
            end_index = len(all_articles_df)
        
        threadx = grabApiThread(j, f'thread_{start_index}_{end_index}', start_index, end_index)
        print(threadx)
        threadx.start()
        threads.append(threadx)
    for t in threads:
        t.join()

    with open(f'{origin_folder}/article_plumx_{i}.json', "w+") as dump_f:
        dump_f.write(json.dumps(plumx_info))

## Generate Plumx Ids

In [3]:
def grab_plumx_ids(i):
    with open(f'{origin_folder}/article_plumx_{i}.json', "r") as f:
        plumx_info = json.load(f)

    df = pd.DataFrame(columns=['DI', 'plumx_id'])
    for doi, item in plumx_info.items():
        if (doi == '') or (str(doi).lower() == 'nan') or (item == None) or ('id' not in item.keys()):
            continue
        dfx = pd.DataFrame(columns=['DI', 'plumx_id'])
        dfx['DI'] = [doi]
        dfx['plumx_id'] = [item['id']]
        df = df.append(dfx, ignore_index=True)
    
    if len(df) % 100 == 0:
        print(len(df))

    df.to_csv(f'{origin_folder}/article_plumx_id_{i}.csv', index=False)

class grabPlumxIdThread(threading.Thread):
    def __init__(self, threadID, name, i):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.i = i
    def run(self):
        grab_plumx_ids(self.i)

In [ ]:
%%time
# threads = []
# for i in range(0, 20):
#     threadx = grabPlumxIdThread(i, f'thread_{i}', i)
#     threadx.start()
#     threads.append(threadx)

# for t in threads:
#     t.join()

for i in range(0, 20):
    print('xxx:', i)
    grab_plumx_ids(i)

## Grab Plumx Tweets Info

In [3]:
def grab_plumx_tweets(i):
    df = pd.read_csv(f'{origin_folder}/article_plumx_id_{i}.csv')
    plumx_tweets_info = {}
    for index, row in df.iterrows():
        if plumx_tweets_info.__contains__(row['DI']):
            continue
        if index % 100 == 0:
            print(index)
        res = grab_from_url_json(f'https://plu.mx/api/v1/artifact/id/{row["plumx_id"]}/metricsEvidence/twitter?offset=0&size=100000')
        plumx_tweets_info[row['DI']] = res

    with open(f'{origin_folder}/article_tweets_plumx_{i}.json', "w+") as dump_f:
        dump_f.write(json.dumps(plumx_tweets_info))

class grabPlumxTweetsThread(threading.Thread):
    def __init__(self, threadID, name, i):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.i = i
    def run(self):
        grab_plumx_tweets(self.i)



In [4]:
%%time
threads = []
for i in range(0, 2):
    for j in range(0, 10):
        threadx = grabPlumxTweetsThread(i, f'thread_{i * 10 + j}', i * 10 + j)
        print(threadx)
        threadx.start()
        threads.append(threadx)

    for t in threads:
        t.join()

# for i in range(0, 20):
#     print('xxx:', i)
#     grab_plumx_tweets(i)

plu.mx', port=443): Read timed out. (read timeout=10)
37800
37800
37900
38000
37900
37900
38000
37900
37900
37900
37900
37900
ReadTimeout: HTTPSConnectionPool(host='plu.mx', port=443): Read timed out. (read timeout=10)
38000
38100
38000
38000
38100
38000
38000
38000
ReadTimeout: HTTPSConnectionPool(host='plu.mx', port=443): Read timed out. (read timeout=10)
38000
38000
38100
38200
38100
38100
38200
38100
38100
38100
ReadTimeout: HTTPSConnectionPool(host='plu.mx', port=443): Read timed out. (read timeout=10)
38100
38100
38200
38300
38200
38200
38300
38200
38200
38200
38200
38200
38300
38400
38300
38300
38400
38300
38300
38300
38300
38300
38400
38500
38400
38400
38500
38400
38400
38400
38400
38400
38500
38600
38500
38500
38600
38500
38500
38500
38500
38500
38600
38700
38600
38600
38700
38600
38600
38600
38600
38600
38700
38800
38700
38700
38800
38700
38700
38700
38700
38700
38800
38900
38800
38800
38900
38800
38800
38800
38800
38800
38900
39000
38900
38900
39000
38900
38900
38900
38900
3

In [ ]:
gc.collect()

In [ ]:
article_tweets = pd.read_json(f'{origin_folder}/article_tweets_plumx.json', "r")

df_tweets = pd.DataFrame(columns=['doi', 'ref', 'tweet_id', 'tweet_url'])

In [ ]:
%%time
for doi in article_tweets:
    if type(article_tweets[doi]['twitter']) == float:
        continue
    for tweet in article_tweets[doi]['twitter']['artifacts']:
        if tweet['id'] in df_tweets['tweet_id']:
            continue
        df_tweet = pd.DataFrame(columns=['doi', 'ref', 'tweet_id', 'tweet_url'])
        df_tweet['doi'] = [doi]
        df_tweet['ref'] = [tweet['ref_id']['value']]
        df_tweet['tweet_id'] = [tweet['id']]
        df_tweet['tweet_url'] = [tweet['title']['link']]

        df_tweets = df_tweets.append(df_tweet, ignore_index=True)

In [ ]:
df_tweets.to_csv(f'{origin_folder}/article_tweets_plumx.csv', index=False)

## Grab Plumx Details

In [ ]:
def grab_detail_plumx(df, doi_column):
    # USAGE
    abstruct_views = list()
    full_text_views = list()
    link_click_count = list()
    link_outs = list()

    # CAPTURE
    exports_saves = list()
    reader_count_mendeley = list()
    reader_count_citeulike = list()

    # CITATION
    cited_by_count_scopus = list()
    cited_by_count_crossref = list()
    cited_by_count_pubmed = list()

    # SOCIAL MEDIA
    twitter = list()
    facebook = list()

    # MENTION
    news = list()
    blogs = list()
    reference_count_wikipedia = list()
    comment_count_reddit = list()
    mention_qa_site_mentions = list()

    a_list = [abstruct_views, full_text_views, link_click_count, link_outs, exports_saves,
              reader_count_mendeley, reader_count_citeulike, cited_by_count_scopus,
              cited_by_count_crossref, cited_by_count_pubmed, twitter, facebook, news, blogs,
              reference_count_wikipedia, comment_count_reddit, mention_qa_site_mentions]
    b_list = ['abstruct_views', 'full_text_views', 'link_click_count', 'link_outs', 'exports_saves',
              'reader_count_mendeley', 'reader_count_citeulike', 'cited_by_count_scopus',
              'cited_by_count_crossref', 'cited_by_count_pubmed', 'tweets', 'facebook', 'news', 'blogs',
              'reference_count_wikipedia', 'comment_count_reddit', 'mention_qa_site_mentions']

    for index, doi in enumerate(df[doi_column]):
        if index % 100 == 0:
            print(index)

        abstruct_views_count, full_text_views_count, link_click_count_count, link_outs_count, \
        exports_saves_count, reader_count_mendeley_count, reader_count_citeulike_count, \
        cited_by_count_scopus_count, cited_by_count_crossref_count, \
        cited_by_count_pubmed_count, twitter_count, facebook_count, news_count, blogs_count, \
        reference_count_wikipedia_count, comment_count_reddit_count, mention_qa_site_mentions_count \
            = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

        res = grab_from_url_json('https://plu.mx/api/v1/artifact/doi/' + str(doi))
        if res is not None:
            if 'sort_count' in res:
                sort_count = res['sort_count']

                # USAGE
                if 'usage' in sort_count:
                    counts = sort_count['usage']['count_types']
                    for item in counts:
                        if item['name'] == 'ABSTRACT_VIEWS':
                            abstruct_views_count = item['total']
                        elif item['name'] == 'FULL_TEXT_VIEWS':
                            full_text_views_count = item['total']
                        elif item['name'] == 'LINK_CLICK_COUNT':
                            link_click_count_count = item['total']
                        elif item['name'] == 'LINK_OUTS':
                            link_outs_count = item['total']

                # CAPTURE
                if 'capture' in sort_count:
                    counts = sort_count['capture']['count_types']
                    for item in counts:
                        if item['name'] == 'EXPORTS_SAVES':
                            exports_saves_count += item['total']
                        elif item['name'] == 'READER_COUNT':
                            for source in item['sources']:
                                if source['name'] == 'Mendeley':
                                    reader_count_mendeley_count += source['total']
                                elif source['name'] == 'CiteULike':
                                    reader_count_citeulike_count += source['total']

                # CITATION
                if 'citation' in sort_count:
                    counts = sort_count['citation']['count_types']
                    for item in counts:
                        if item['name'] == 'Scopus':
                            cited_by_count_scopus_count = item['total']
                        elif item['name'] == 'CrossRef':
                            cited_by_count_crossref_count = item['total']
                        elif item['name'] == 'PubMed':
                            cited_by_count_pubmed_count = item['total']

                # SOCIAL MEDIA
                if 'social_media' in sort_count:
                    counts = sort_count['social_media']['count_types']
                    for item in counts:
                        if item['name'] == 'TWEET_COUNT':
                            twitter_count = item['total']
                        elif item['name'] == 'FACEBOOK_COUNT':
                            facebook_count = item['total']

                # MENTION
                if 'mention' in sort_count:
                    counts = sort_count['mention']['count_types']
                    for item in counts:
                        if item['name'] == 'NEWS_COUNT':
                            news_count = item['total']
                        elif item['name'] == 'ALL_BLOG_COUNT':
                            blogs_count = item['total']
                        elif item['name'] == 'REFERENCE_COUNT':
                            for source in item['sources']:
                                if source['name'] == 'Wikipedia':
                                    reference_count_wikipedia_count += source['total']
                        elif item['name'] == 'COMMENT_COUNT':
                            for source in item['sources']:
                                if source['name'] == 'Reddit':
                                    comment_count_reddit_count += source['total']
                        elif item['name'] == 'QA_SITE_MENTIONS':
                            mention_qa_site_mentions_count = item['total']

        c_list = [abstruct_views_count, full_text_views_count, link_click_count_count, link_outs_count,
                  exports_saves_count, reader_count_mendeley_count, reader_count_citeulike_count,
                  cited_by_count_scopus_count, cited_by_count_crossref_count,
                  cited_by_count_pubmed_count, twitter_count, facebook_count, news_count, blogs_count,
                  reference_count_wikipedia_count, comment_count_reddit_count,
                  mention_qa_site_mentions_count]

        for i in range(0, len(a_list)):
            a_list[i].append(c_list[i])

    for i in range(0, len(a_list)):
        print(len(a_list[i]))
        df[b_list[i]] = a_list[i]

    return df


In [ ]:
# df = pd.read_csv(f'{origin_folder}/all.csv', usecols=['SO', 'DI'])
df = pd.read_csv(f'{origin_folder}/article_tweets_altmetric.csv', usecols=['doi'])
df = df.drop_duplicates()
print(len(df))
df = grab_detail_plumx(df, 'doi')
df.to_csv(f'{origin_folder}/all_plumx_detail.csv', index=False)

In [ ]:
len(df)

In [5]:
def order_detail_plumx(index):
    with open(f'{origin_folder}/article_plumx_{index}.json') as f:
        articles_details = json.load(f)

    b_list = ['doi', 'abstruct_views', 'full_text_views', 'link_click_count', 'link_outs', 'exports_saves',
              'reader_count_mendeley', 'reader_count_citeulike', 'cited_by_count_scopus',
              'cited_by_count_crossref', 'cited_by_count_pubmed', 'tweets', 'facebook', 'news', 'blogs',
              'reference_count_wikipedia', 'comment_count_reddit', 'mention_qa_site_mentions']
    df_details = pd.DataFrame(columns=b_list)

    doi_count = 0
    for doi in articles_details:
        doi_count += 1
        dfx = pd.DataFrame(columns=b_list)
        if doi_count % 100 == 0:
            print(doi_count)
        
        abstruct_views_count, full_text_views_count, link_click_count_count, link_outs_count, \
        exports_saves_count, reader_count_mendeley_count, reader_count_citeulike_count, \
        cited_by_count_scopus_count, cited_by_count_crossref_count, \
        cited_by_count_pubmed_count, twitter_count, facebook_count, news_count, blogs_count, \
        reference_count_wikipedia_count, comment_count_reddit_count, mention_qa_site_mentions_count \
            = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
        
        res = articles_details[doi]
        if res is not None:
            if 'sort_count' in res:
                sort_count = res['sort_count']

                # USAGE
                if 'usage' in sort_count:
                    counts = sort_count['usage']['count_types']
                    for item in counts:
                        if item['name'] == 'ABSTRACT_VIEWS':
                            abstruct_views_count = item['total']
                        elif item['name'] == 'FULL_TEXT_VIEWS':
                            full_text_views_count = item['total']
                        elif item['name'] == 'LINK_CLICK_COUNT':
                            link_click_count_count = item['total']
                        elif item['name'] == 'LINK_OUTS':
                            link_outs_count = item['total']

                # CAPTURE
                if 'capture' in sort_count:
                    counts = sort_count['capture']['count_types']
                    for item in counts:
                        if item['name'] == 'EXPORTS_SAVES':
                            exports_saves_count += item['total']
                        elif item['name'] == 'READER_COUNT':
                            for source in item['sources']:
                                if source['name'] == 'Mendeley':
                                    reader_count_mendeley_count += source['total']
                                elif source['name'] == 'CiteULike':
                                    reader_count_citeulike_count += source['total']

                # CITATION
                if 'citation' in sort_count:
                    counts = sort_count['citation']['count_types']
                    for item in counts:
                        if item['name'] == 'Scopus':
                            cited_by_count_scopus_count = item['total']
                        elif item['name'] == 'CrossRef':
                            cited_by_count_crossref_count = item['total']
                        elif item['name'] == 'PubMed':
                            cited_by_count_pubmed_count = item['total']

                # SOCIAL MEDIA
                if 'social_media' in sort_count:
                    counts = sort_count['social_media']['count_types']
                    for item in counts:
                        if item['name'] == 'TWEET_COUNT':
                            twitter_count = item['total']
                        elif item['name'] == 'FACEBOOK_COUNT':
                            facebook_count = item['total']

                # MENTION
                if 'mention' in sort_count:
                    counts = sort_count['mention']['count_types']
                    for item in counts:
                        if item['name'] == 'NEWS_COUNT':
                            news_count = item['total']
                        elif item['name'] == 'ALL_BLOG_COUNT':
                            blogs_count = item['total']
                        elif item['name'] == 'REFERENCE_COUNT':
                            for source in item['sources']:
                                if source['name'] == 'Wikipedia':
                                    reference_count_wikipedia_count += source['total']
                        elif item['name'] == 'COMMENT_COUNT':
                            for source in item['sources']:
                                if source['name'] == 'Reddit':
                                    comment_count_reddit_count += source['total']
                        elif item['name'] == 'QA_SITE_MENTIONS':
                            mention_qa_site_mentions_count = item['total']

        c_list = [doi, abstruct_views_count, full_text_views_count, link_click_count_count, link_outs_count,
                  exports_saves_count, reader_count_mendeley_count, reader_count_citeulike_count,
                  cited_by_count_scopus_count, cited_by_count_crossref_count,
                  cited_by_count_pubmed_count, twitter_count, facebook_count, news_count, blogs_count,
                  reference_count_wikipedia_count, comment_count_reddit_count,
                  mention_qa_site_mentions_count]

        for i in range(0, len(c_list)):
            dfx[b_list[i]] = [c_list[i]]
        
        df_details = df_details.append(dfx)

    df_details.to_csv(f'{origin_folder}/article_plumx_{index}.csv', index=False)

    return df_details

In [6]:
%%time

for i in range(0, 20):
    # threadx = threading.Thread(target=order_detail_plumx, args=(i,))
    # threadx.start()
    order_detail_plumx(i)
    

0
26600
26700
26800
26900
27000
27100
27200
27300
27400
27500
27600
27700
27800
27900
28000
28100
28200
28300
28400
28500
28600
28700
28800
28900
29000
29100
29200
29300
29400
29500
29600
29700
29800
29900
30000
30100
30200
30300
30400
30500
30600
30700
30800
30900
31000
31100
31200
31300
31400
31500
31600
31700
31800
31900
32000
32100
32200
32300
32400
32500
32600
32700
32800
32900
33000
33100
33200
33300
33400
33500
33600
33700
33800
33900
34000
34100
34200
34300
34400
34500
34600
34700
34800
34900
35000
35100
35200
35300
35400
35500
35600
35700
35800
35900
36000
36100
36200
36300
36400
36500
36600
36700
36800
36900
37000
37100
37200
37300
37400
37500
37600
37700
37800
37900
38000
38100
38200
38300
38400
38500
38600
38700
38800
38900
39000
39100
39200
39300
39400
39500
39600
39700
39800
39900
40000
40100
40200
40300
40400
40500
40600
40700
40800
40900
41000
41100
41200
41300
41400
41500
41600
41700
41800
41900
42000
42100
42200
42300
42400
42500
42600
42700
42800
42900
43000
43100
43